#Movie Recommedation

Final Project PKT
Nasywa Befiputri - 2203015044

##library

In [17]:
!pip install scikit-surprise

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
import numpy as np
import pandas as pd

##load dataset



In [19]:

url = "https://raw.githubusercontent.com/nasywabefi/Collage_PerngantarPecerdasanTiruan/main/finalsubmission/netflix_data.csv"
data = pd.read_csv(url)

##cek data

In [20]:
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [22]:
data.release_year.view()

<ipython-input-22-39d7920a79d8>:1: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  data.release_year.view()


,release_year
0,2020
1,2021
2,2021
3,2021
4,2021
...,...
8802,2007
8803,2018
8804,2009
8805,2006


##bersihin data yang kosong

In [23]:
data.fillna({
    'director': '',
    'cast': '',
    'description': '',
    'rating': 'Unknown',
    'duration': '0 min',
    'country': '',
    'date_added': ''
}, inplace=True)

mensimulasikan data pengguna (user_id)

In [24]:
data['user_id'] = np.random.randint(1, 1001, size=len(data))

 Kategori Rating ke Nilai Numerik

In [25]:
rating_map = {
    'Unknown': 0, 'G': 1, 'PG': 2, 'PG-13': 3, 'R': 4, 'NC-17': 5
}

data['rating'] = data['rating'].map(rating_map).fillna(0).astype(int)


Content Based FIltering

In [26]:
# TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Combine description, director, cast, and listed_in as content features
content_matrix = tfidf.fit_transform(
    data['description'] + " " + data['director'] + " " + data['cast'] + " " + data['listed_in']
)


content_similarity = cosine_similarity(content_matrix)

Collaborative Filtering

In [27]:
# Collaborative Filtering

# Menginisialisasi Reader untuk memproses data rating
reader = Reader(rating_scale=(1, 5))


# Membuat dataset dari DataFrame
rating_data = Dataset.load_from_df(data[['user_id', 'title', 'rating']], reader)

# Membagi dataset menjadi data latih dan uji
trainset, testset = train_test_split(rating_data, test_size=0.2)

# Menginisialisasi model Singular Value Decomposition (SVD)
svd_model = SVD()

# Melatih model SVD dengan data latih
svd_model.fit(trainset)



Hybrid

In [28]:
# Hybrid Recommendation System
class HybridFlixHub:
    def __init__(self, content_sim, svd_model, data):
        # Inisialisasi dengan matriks similarity berbasis konten, model SVD untuk collaborative filtering, dan data film.
        self.content_sim = content_sim  # Matriks similarity berbasis konten
        self.svd_model = svd_model      # Model collaborative filtering (SVD)
        self.data = data                # Dataframe berisi informasi film dan metadata

    def recommend(self, title, alpha=0.5, beta=0.5, n=10):

        # Cari indeks film yang cocok dengan judul input
        idx = self.data[self.data['title'].str.contains(title, case=False, na=False)].index
        if len(idx) == 0:  # kalau ga ketemu film dengan judul tersebut, kembalikan list kosong
            return []
        content_scores = self.content_sim[idx[0]]  # Ambil skor similaritas berbasis konten untuk film tersebut

        # Hitung skor collaborative filtering menggunakan model SVD
        unique_titles = self.data['title'].unique()  # Dapatkan semua judul film unik
        collaborative_scores = [
            (movie, self.svd_model.predict(1, movie).est)  # Prediksi skor untuk user_id=1 (dummy user)
            for movie in unique_titles
        ]
        collaborative_scores = {movie: score for movie, score in collaborative_scores}  # Konversi ke dictionary

        # Gabungkan skor content-based dan collaborative untuk menghasilkan hybrid score
        hybrid_scores = []
        for i, movie in enumerate(unique_titles):
            content_score = content_scores[i]  # Ambil skor similaritas berbasis konten
            collaborative_score = collaborative_scores.get(movie, 0)  # Ambil skor collaborative atau 0 jika tidak ada
            # Hitung skor hybrid sebagai kombinasi linier dari content dan collaborative scores
            hybrid_score = alpha * content_score + beta * collaborative_score
            hybrid_scores.append((movie, hybrid_score))

        # Urutkan film berdasarkan skor hybrid secara descending, lalu ambil n rekomendasi teratas
        hybrid_scores = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)[:n]
        return [rec[0] for rec in hybrid_scores]  # Kembalikan hanya judul film


In [29]:
# Create Hybrid System

# Menginisialisasi sistem rekomendasi hybrid menggunakan kelas HybridFlixHub
hybrid_system = HybridFlixHub(content_similarity, svd_model, data)


Outcome

In [33]:
# Meminta pengguna untuk memasukkan judul film yang akan dijadikan referensi rekomendasi.
movie_title = input("Masukkan judul film untuk mendapatkan rekomendasi: ")

# Menggunakan sistem rekomendasi hybrid untuk mendapatkan daftar rekomendasi berdasarkan judul film yang diberikan.
recommendations = hybrid_system.recommend(movie_title)

# Display Recommendations
if recommendations:
    # Jika terdapat rekomendasi, tampilkan hasilnya.
    print(f"\nRekomendasi untuk film '{movie_title}':")
    for idx, rec in enumerate(recommendations, start=1):
        # Menampilkan daftar rekomendasi satu per satu dengan format terurut.
        print(f"{idx}. {rec}")
else:
    # Jika tidak ada film yang cocok atau mirip dengan judul yang diberikan, beri pesan pemberitahuan.
    print(f"Tidak ditemukan film yang mirip dengan judul '{movie_title}'")


Masukkan judul film untuk mendapatkan rekomendasi: blood & water

Rekomendasi untuk film 'blood & water':
1. Blood & Water
2. Diamond City
3. About Time
4. Kings of Jo'Burg
5. Shirkers
6. Sniper: Ghost Shooter
7. Fun Mom Dinner
8. Dive Club
9. The Humanity Bureau
10. Get Even


In [ ]:
from surprise import accuracy

# Evaluasi model menggunakan dataset uji
predictions = svd_model.test(testset)

# Menghitung metrik evaluasi
mae = accuracy.mae(predictions, verbose=True)  # Mean Absolute Error
rmse = accuracy.rmse(predictions, verbose=True)  # Root Mean Square Error

# Maksimum rating dalam dataset
max_rating = 5  # Karena rating scale adalah (1-5)

# Menghitung persentase akurasi
mae_accuracy = (1 - mae / max_rating) * 100
rmse_accuracy = (1 - rmse / max_rating) * 100

print(f"\nPersentase Akurasi Berdasarkan MAE: {mae_accuracy:.2f}%")
print(f"Persentase Akurasi Berdasarkan RMSE: {rmse_accuracy:.2f}%")


MAE:  1.2337
RMSE: 1.3766

Persentase Akurasi Berdasarkan MAE: 75.33%
Persentase Akurasi Berdasarkan RMSE: 72.47%
